In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
dataset = pd.read_csv("heart.csv")
dataset.head()


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
dataset.dtypes


age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal          int64
target        int64
dtype: object

In [4]:
X = dataset.iloc[:, :-1]
X.head()


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2


In [5]:
y = dataset.iloc[:, [-1]]
y.head()


,target
0,1
1,1
2,1
3,1
4,1


In [6]:
X_ohe = pd.get_dummies(X, columns=["cp", "restecg", "slope", "thal"])


In [7]:
X_ohe.columns


Index(['age', 'sex', 'trestbps', 'chol', 'fbs', 'thalach', 'exang', 'oldpeak',
       'ca', 'cp_0', 'cp_1', 'cp_2', 'cp_3', 'restecg_0', 'restecg_1',
       'restecg_2', 'slope_0', 'slope_1', 'slope_2', 'thal_0', 'thal_1',
       'thal_2', 'thal_3'],
      dtype='object')

In [11]:
X_ohe = X_ohe.drop(['cp_0', 'restecg_0', 'slope_0', 'thal_0'], axis = 1)

In [12]:
X_ohe.head()

,age,sex,trestbps,chol,fbs,thalach,exang,oldpeak,ca,cp_1,cp_2,cp_3,restecg_1,restecg_2,slope_1,slope_2,thal_1,thal_2,thal_3
0,63,1,145,233,1,150,0,2.3,0,0,0,1,0,0,0,0,1,0,0
1,37,1,130,250,0,187,0,3.5,0,0,1,0,1,0,0,0,0,1,0
2,41,0,130,204,0,172,0,1.4,0,1,0,0,0,0,0,1,0,1,0
3,56,1,120,236,0,178,0,0.8,0,1,0,0,1,0,0,1,0,1,0
4,57,0,120,354,0,163,1,0.6,0,0,0,0,1,0,0,1,0,1,0


In [13]:
X_ohe.shape

(303, 19)

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_ohe, y, test_size=0.3, random_state=0
)


In [15]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)


In [16]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout


In [17]:
classifier = Sequential()
classifier.add(keras.Input(shape=(19,)))
classifier.add(Dense(1024, activation="relu"))
classifier.add(Dropout(0.15))
classifier.add(Dense(512, activation="relu"))
classifier.add(Dropout(0.15))
classifier.add(Dense(512, activation="relu"))
# classifier.add(Dropout(0.1))
classifier.add(Dense(1, activation="sigmoid"))
classifier.compile(
    optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]
)

In [18]:
classifier.fit(X_train, y_train,validation_data=(X_test, y_test),
    batch_size=25,
    epochs=30,)


Epoch 1/30
9/9 [==============================] - 2s 22ms/step - loss: 0.4792 - accuracy: 0.7877 - val_loss: 0.4727 - val_accuracy: 0.8132
Epoch 2/30
9/9 [==============================] - 0s 8ms/step - loss: 0.3740 - accuracy: 0.8585 - val_loss: 0.5387 - val_accuracy: 0.7802
Epoch 3/30
9/9 [==============================] - 0s 7ms/step - loss: 0.3112 - accuracy: 0.8538 - val_loss: 0.4013 - val_accuracy: 0.8462
Epoch 4/30
9/9 [==============================] - 0s 7ms/step - loss: 0.2393 - accuracy: 0.9009 - val_loss: 0.4600 - val_accuracy: 0.8242
Epoch 5/30
9/9 [==============================] - 0s 7ms/step - loss: 0.2172 - accuracy: 0.9151 - val_loss: 0.4665 - val_accuracy: 0.8242
Epoch 6/30
9/9 [==============================] - 0s 8ms/step - loss: 0.1916 - accuracy: 0.9198 - val_loss: 0.5308 - val_accuracy: 0.8132
Epoch 7/30
9/9 [==============================] - 0s 7ms/step - loss: 0.1279 - accuracy: 0.9481 - val_loss: 0.5571 - val_accuracy: 0.8242
Epoch 8/30
9/9 [=================

In [19]:
from sklearn.metrics import accuracy_score


In [20]:
y_pred = classifier.predict(X_test)
y_pred = y_pred > 0.5


In [21]:
acc = accuracy_score(y_true=y_test, y_pred=y_pred) * 100
print('Accuracy score:', acc)


Accuracy score: 83.51648351648352
